<a href="https://colab.research.google.com/github/heyronith/lazarus/blob/main/mPox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Set Up

In [ ]:
import zipfile
import os

# Extract the dataset
zip_path = '/content/Monkeypox Skin Image Dataset.zip'  # Adjust this path if necessary
extract_path = '/content'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully.")

Dataset extracted successfully.


In [ ]:
import os

extract_path = '/content/Monkeypox Skin Image Dataset'
print("Contents of the extracted folder:")
print(os.listdir(extract_path))

# Check if there are any subdirectories
for item in os.listdir(extract_path):
    item_path = os.path.join(extract_path, item)
    if os.path.isdir(item_path):
        print(f"Contents of {item}:")
        print(os.listdir(item_path))

Contents of the extracted folder:
['Measles', 'Chickenpox', 'Normal', 'Monkeypox']
Contents of Measles:
['measles75.png', 'measles5.png', 'measles13.png', 'measles21.png', 'measles42.png', 'measles85.png', 'measles20.png', 'measles37.png', 'measles50.png', 'measles73.png', 'measles63.png', 'measles10.png', 'measles77.png', 'measles16.png', 'measles55.png', 'measles33.png', 'measles60.png', 'measles78.png', 'measles58.png', 'measles67.png', 'measles88.png', 'measles2.png', 'measles32.png', 'measles82.png', 'measles83.png', 'measles28.png', 'measles46.png', 'measles48.png', 'measles24.png', 'measles69.png', 'measles15.png', 'measles4.png', 'measles52.png', 'measles29.png', 'measles80.png', 'measles62.png', 'measles25.png', 'measles68.png', 'measles9.png', 'measles31.png', 'measles26.png', 'measles47.png', 'measles66.png', 'measles19.png', 'measles89.png', 'measles7.png', 'measles49.png', 'measles1.png', 'measles74.png', 'measles81.png', 'measles44.png', 'measles72.png', 'measles11.png', 

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import albumentations as A
from PIL import Image
import numpy as np

def create_dataset_df(data_dir):
    image_paths = []
    labels = []

    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        if os.path.isdir(class_path):
            for image_file in os.listdir(class_path):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(class_folder, image_file))
                    labels.append(class_folder)

    df = pd.DataFrame({'image_path': image_paths, 'label': labels})
    return df

# Create the original dataset DataFrame
extract_path = '/content/Monkeypox Skin Image Dataset'
df_original = create_dataset_df(extract_path)

print("Original dataset created.")
print(f"Dataset size: {len(df_original)}")
print("\nClass distribution:")
print(df_original['label'].value_counts())

# Split the data into train and test sets
train_df, test_df = train_test_split(df_original, test_size=0.2, stratify=df_original['label'], random_state=42)

print("\nTraining set size:", len(train_df))
print("Test set size:", len(test_df))

# Create augmented dataset
def get_train_augmentations(height=224, width=224):
    return A.Compose([
        A.RandomRotate90(p=0.5),
        A.Rotate(limit=45, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomScale(scale_limit=(0.8, 1.25), p=0.5),
        A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(0.1, 2), contrast_limit=(0.5, 2), p=0.5),
        A.HueSaturationValue(hue_shift_limit=0.5, sat_shift_limit=(0.2, 3), val_shift_limit=0, p=0.5),
        A.Resize(height=height, width=width),
    ])

def augment_image(image_path, augmentations):
    image = Image.open(image_path).convert('RGB')
    image = np.array(image)
    augmented = augmentations(image=image)
    return augmented['image']

augmentations = get_train_augmentations()
augmented_images = []
augmented_labels = []

for _, row in train_df.iterrows():
    img_path = os.path.join(extract_path, row['image_path'])
    label = row['label']

    # Add original image
    augmented_images.append(img_path)
    augmented_labels.append(label)

    # Add augmented images
    for _ in range(11):  # Create 11 augmented versions of each image
        aug_img = augment_image(img_path, augmentations)
        augmented_images.append(aug_img)
        augmented_labels.append(label)

# Create augmented dataset DataFrame
df_augmented = pd.DataFrame({'image_path': augmented_images, 'label': augmented_labels})

print("\nAugmented dataset created.")
print(f"Augmented dataset size: {len(df_augmented)}")
print("\nAugmented class distribution:")
print(df_augmented['label'].value_counts())

# Save the datasets
train_df.to_csv('train_original.csv', index=False)
test_df.to_csv('test_original.csv', index=False)
df_augmented.to_csv('train_augmented.csv', index=False)

print("\nDatasets saved as CSV files.")

Original dataset created.
Dataset size: 770

Class distribution:
label
Normal        293
Monkeypox     279
Chickenpox    107
Measles        91
Name: count, dtype: int64

Training set size: 616
Test set size: 154

Augmented dataset created.
Augmented dataset size: 7392

Augmented class distribution:
label
Normal        2808
Monkeypox     2676
Chickenpox    1032
Measles        876
Name: count, dtype: int64

Datasets saved as CSV files.


# Phase1: Training on Original Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from PIL import Image
import pandas as pd
import numpy as np
import os

# Custom Model Definition
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class FPN(nn.Module):
    def __init__(self, in_channels_list, out_channels):
        super(FPN, self).__init__()
        self.lateral_convs = nn.ModuleList([
            nn.Conv2d(in_channels, out_channels, kernel_size=1)
            for in_channels in in_channels_list
        ])
        self.fpn_convs = nn.ModuleList([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
            for _ in in_channels_list
        ])

    def forward(self, features):
        last_feature = self.lateral_convs[-1](features[-1])
        fpn_features = [self.fpn_convs[-1](last_feature)]

        for feature, lateral_conv, fpn_conv in zip(
            features[-2::-1], self.lateral_convs[-2::-1], self.fpn_convs[-2::-1]
        ):
            lateral = lateral_conv(feature)
            feat_shape = lateral.shape[-2:]
            top_down = F.interpolate(last_feature, size=feat_shape, mode='nearest')
            last_feature = lateral + top_down
            fpn_features.append(fpn_conv(last_feature))

        return fpn_features[::-1]

class CustomModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomModel, self).__init__()
        self.efficientnet = models.efficientnet_v2_s(pretrained=True)

        # Get the number of features from the last layer of EfficientNetV2
        num_features = self.efficientnet.classifier[1].in_features

        # Remove the classifier
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(num_features, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Initialize the model
model = CustomModel()
print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# Rest of the code remains the same...

# Custom Dataset
class MonkeypoxDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.class_to_idx = {'Normal': 0, 'Monkeypox': 1, 'Chickenpox': 2, 'Measles': 3}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.class_to_idx[self.data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
train_dataset = MonkeypoxDataset('train_original.csv', '/content/Monkeypox Skin Image Dataset', transform=transform)
test_dataset = MonkeypoxDataset('test_original.csv', '/content/Monkeypox Skin Image Dataset', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Initialize the model
model = CustomModel()

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# Loss function (Focal Loss)
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        return focal_loss.mean()

criterion = FocalLoss()

# Optimizer and learning rate scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# Training function
def train(model, train_loader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        if i % 10 == 0:  # Print every 10 mini-batches
            print(f'Batch {i}, Loss: {loss.item():.4f}')

    scheduler.step()
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

# Evaluation function
def evaluate(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # For AUC, we need probability scores
    all_probs = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())

    all_probs = np.array(all_probs)
    auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='weighted')

    return accuracy, precision, recall, f1, auc

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 100
best_accuracy = 0.0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, scheduler, device)
    accuracy, precision, recall, f1, auc = evaluate(model, test_loader, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

print("Training completed.")

# Load the best model and evaluate
model.load_state_dict(torch.load('best_model.pth'))
final_accuracy, final_precision, final_recall, final_f1, final_auc = evaluate(model, test_loader, device)

print("Final Results on Original Dataset:")
print(f"Accuracy: {final_accuracy:.4f}")
print(f"Precision: {final_precision:.4f}")
print(f"Recall: {final_recall:.4f}")
print(f"F1-score: {final_f1:.4f}")
print(f"AUC: {final_auc:.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 125MB/s]


Model initialized with 20965716 parameters
Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda
Epoch 1/100
Batch 0, Loss: 0.7811
Batch 10, Loss: 0.5882
Train Loss: 0.5290
Test Accuracy: 0.6494, Precision: 0.7196, Recall: 0.6494, F1: 0.6579, AUC: 0.8610
Epoch 2/100
Batch 0, Loss: 0.2818
Batch 10, Loss: 0.4154
Train Loss: 0.2939
Test Accuracy: 0.8506, Precision: 0.8544, Recall: 0.8506, F1: 0.8434, AUC: 0.9685
Epoch 3/100
Batch 0, Loss: 0.0934
Batch 10, Loss: 0.2270
Train Loss: 0.2276
Test Accuracy: 0.8052, Precision: 0.8430, Recall: 0.8052, F1: 0.8000, AUC: 0.9708
Epoch 4/100
Batch 0, Loss: 0.1624
Batch 10, Loss: 0.0722
Train Loss: 0.1489
Test Accuracy: 0.9351, Precision: 0.9353, Recall: 0.9351, F1: 0.9346, AUC: 0.9748
Epoch 5/100
Batch 0, Loss: 0.0094
Batch 10, Loss: 0.0689
Train Loss: 0.1452
Test Accuracy: 0.8701, Precision: 0.9118, Recall: 0.8701, F1: 0.8784, AUC: 0.9841
Epoch 6/100
Batch 0, Loss: 0.0751
Batch 10, Loss: 0.0960
Train Loss: 0.0669
Test Accuracy: 0.9221, Precision: 0.9276, Recall: 0.9221, F1: 0.9214, AUC: 0.9906
Epoch 7/100
Batch 0, Loss: 0.0284

# Phase2: Augmentation

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from tqdm import tqdm

def augment_and_save(input_dir, output_dir, num_augmentations=11):
    # Define augmentations similar to the paper
    aug_transform = A.Compose([
        A.RandomRotate90(p=0.5),
        A.Rotate(limit=45, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomScale(scale_limit=(0.8, 1.25), p=0.5),
        A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(0.1, 2), contrast_limit=(0.5, 2), p=0.5),
        A.HueSaturationValue(hue_shift_limit=0.5, sat_shift_limit=(0.2, 3), val_shift_limit=0, p=0.5),
    ])

    class_counts = {}

    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        output_class_dir = os.path.join(output_dir, class_name)
        os.makedirs(output_class_dir, exist_ok=True)

        class_counts[class_name] = 0

        for img_name in tqdm(os.listdir(class_dir), desc=f"Augmenting {class_name}"):
            img_path = os.path.join(class_dir, img_name)
            image = np.array(Image.open(img_path).convert('RGB'))

            # Save original image
            output_path = os.path.join(output_class_dir, img_name)
            Image.fromarray(image).save(output_path)
            class_counts[class_name] += 1

            # Generate augmented images
            for i in range(num_augmentations):
                aug_image = aug_transform(image=image)['image']
                aug_name = f"{os.path.splitext(img_name)[0]}_aug_{i}{os.path.splitext(img_name)[1]}"
                output_path = os.path.join(output_class_dir, aug_name)
                Image.fromarray(aug_image).save(output_path)
                class_counts[class_name] += 1

    print("\nAugmentation completed. Class distribution:")
    for class_name, count in class_counts.items():
        print(f"{class_name}: {count}")

    return class_counts

def create_csv(augmented_dir, output_csv):
    data = []
    for class_name in os.listdir(augmented_dir):
        class_dir = os.path.join(augmented_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_name, img_name)
            data.append([img_path, class_name])

    df = pd.DataFrame(data, columns=['image_path', 'label'])
    df = df.sample(frac=1).reset_index(drop=True)  # Shuffle the dataset
    df.to_csv(output_csv, index=False)
    print(f"\nCSV file created: {output_csv}")
    print(f"Total samples: {len(df)}")

# Usage
input_dir = '/content/Monkeypox Skin Image Dataset'
output_dir = '/content/MSID_dataset/augmented_train'
output_csv = '/content/MSID_dataset/augmented_train.csv'

class_counts = augment_and_save(input_dir, output_dir)
create_csv(output_dir, output_csv)

Augmenting Monkeypox: 100%|██████████| 279/279 [01:55<00:00,  2.42it/s]


Augmentation completed. Class distribution:
Measles: 1092
Chickenpox: 1284
Normal: 3516
Monkeypox: 3348

CSV file created: /content/MSID_dataset/augmented_train.csv
Total samples: 9240


# Training Augmented Data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
import torch.nn.functional as F
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

class MonkeypoxDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.class_to_idx = {'Normal': 0, 'Monkeypox': 1, 'Chickenpox': 2, 'Measles': 3}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.class_to_idx[self.data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label

class CustomModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomModel, self).__init__()
        self.efficientnet = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.IMAGENET1K_V1)

        # Get the number of features from the last layer of EfficientNetV2
        num_features = self.efficientnet.classifier[1].in_features

        # Remove the classifier
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(num_features, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and dataloaders
train_dataset = MonkeypoxDataset('/content/MSID_dataset/augmented_train.csv', '/content/MSID_dataset/augmented_train', transform=transform)
test_dataset = MonkeypoxDataset('test_original.csv', '/content/Monkeypox Skin Image Dataset', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Initialize the model
model = CustomModel()
print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer and learning rate scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# Training function
def train(model, train_loader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Batch {i+1}, Loss: {loss.item():.4f}')

    scheduler.step()
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

# Evaluation function
def evaluate(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            probs = F.softmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovr', average='weighted')

    return accuracy, precision, recall, f1, auc

# Training loop
num_epochs = 100
best_accuracy = 0.0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, scheduler, device)
    accuracy, precision, recall, f1, auc = evaluate(model, test_loader, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model_augmented.pth')

print("Training completed.")

# Load the best model and evaluate
model.load_state_dict(torch.load('best_model_augmented.pth'))
final_accuracy, final_precision, final_recall, final_f1, final_auc = evaluate(model, test_loader, device)

print("Final Results on Augmented Dataset:")
print(f"Accuracy: {final_accuracy:.4f}")
print(f"Precision: {final_precision:.4f}")
print(f"Recall: {final_recall:.4f}")
print(f"F1-score: {final_f1:.4f}")
print(f"AUC: {final_auc:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/MSID_dataset/augmented_train.csv'

# Ensemble Model and Preparing for Hugging Face deploy


In [ ]:
import torch
import torch.nn as nn

class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        return (out1 + out2) / 2  # Simple averaging of predictions

# Assuming you have already trained model1 and model2
ensemble_model = EnsembleModel(model1, model2)